# Data Extraction from TMDB API

In [7]:
# Import Libraries 
import pandas as pd
import numpy as np
import os, json, time
from tqdm.notebook import tqdm_notebook 

## User Defined Functions

### get_movie_with_rating Function

In [8]:
# function to get movie raiting from tmdb api
def get_movie_with_rating(movie_id):
    """Adapted from source = https://github.com/celiao/tmdbsimple"""
    # get movie object for current id
    movie = tmdb.Movies(movie_id)
    
    # save the .info .releases dictionaries
    info = movie.info()
    releases = movie.releases()
    
    # loop through countries in releases
    for c in releases['countries']:
        # if country abbreviation==US
        if c['iso_3166_1'] == 'US':
            # save a 'certification' key in info with the certification
            info['certification'] = c['certification']
    
    return info

#### Test get_movie_with_rating Function

Test with The Avengers

In [14]:
test1 = get_movie_with_rating('tt0848228')
test1

{'adult': False,
 'backdrop_path': '/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg',
 'belongs_to_collection': {'id': 86311,
  'name': 'The Avengers Collection',
  'poster_path': '/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg',
  'backdrop_path': '/zuW6fOiusv4X9nnW3paHGfXcSll.jpg'},
 'budget': 220000000,
 'genres': [{'id': 878, 'name': 'Science Fiction'},
  {'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'}],
 'homepage': 'https://www.marvel.com/movies/the-avengers',
 'id': 24428,
 'imdb_id': 'tt0848228',
 'original_language': 'en',
 'original_title': 'The Avengers',
 'overview': 'When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!',
 'popularity': 183.934,
 'poster_path': '/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg',
 'production_companies': [{'id': 420,
   'logo_path

Test with The Notebook 

In [15]:
test2 = get_movie_with_rating('tt0332280')
test2

{'adult': False,
 'backdrop_path': '/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg',
 'belongs_to_collection': None,
 'budget': 29000000,
 'genres': [{'id': 10749, 'name': 'Romance'}, {'id': 18, 'name': 'Drama'}],
 'homepage': 'http://www.newline.com/properties/notebookthe.html',
 'id': 11036,
 'imdb_id': 'tt0332280',
 'original_language': 'en',
 'original_title': 'The Notebook',
 'overview': "An epic love story centered around an older man who reads aloud to a woman with Alzheimer's. From a faded notebook, the old man's words bring to life the story about a couple who is separated by World War II, and is then passionately reunited, seven years later, after they have taken different paths.",
 'popularity': 52.944,
 'poster_path': '/rNzQyW4f8B8cQeg7Dgj3n6eT5k9.jpg',
 'production_companies': [{'id': 12,
   'logo_path': '/5ThIuO93vsk47oexKTSdfKEr7EC.png',
   'name': 'New Line Cinema',
   'origin_country': 'US'},
  {'id': 1565, 'logo_path': None, 'name': 'Avery Pix', 'origin_country': 'US'},
  {'id': 26

The function works as predicted.

### write_json Function

In [16]:
# appends a list of records (new_data) to a json file
# addapted from : https://www.geeksforgeeks.org/append-to-json-file-using-python/

def write_json(new_data, filename):
    with open(filename,'r+') as file:
        # first load existing data into dict
        file_data = json.load(file)
        # choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
            file_data.append(new_data)
        # set file's current position at offset
        file.seek(0)
        # convert back to json
        json.dump(file_data, file)

#### Test write_json Function

In [17]:
# create new json file with empty list
JSON_FILE = 'Data/test_json.json'
with open(JSON_FILE,'w') as f:
    json.dump([],f)
write_json(test1, JSON_FILE)
write_json(test2, JSON_FILE)

In [18]:
# save json file as dataframe and look at info
test_df = pd.read_json(JSON_FILE)
test_df

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,"{'id': 86311, 'name': 'The Avengers Collection...",220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",https://www.marvel.com/movies/the-avengers,24428,tt0848228,en,The Avengers,...,1518815515,143,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some assembly required.,The Avengers,False,7.707,28314,PG-13
1,False,/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg,None,29000000,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",http://www.newline.com/properties/notebookthe....,11036,tt0332280,en,The Notebook,...,115603229,123,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Behind every great love is a great story.,The Notebook,False,7.878,10201,PG-13


The function works as predicted. 

## Extract Data from TMDB

### tmdbsimple Package

In [11]:
!pip install tmdbsimple;
import tmdbsimple as tmdb

In [12]:
# load login credentials using json file in notebook
with open('E:/Dropbox/Coding Dojo/Projects/Project 3/movie_success_predictions/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
# display keys of loaded dict
login.keys()

dict_keys(['api-key'])

In [13]:
# save api-key using tmdbsimple
tmdb.API_KEY = login['api-key']

### Extract data from TMDB API

In [19]:
# define list of years to extract
YEARS_TO_GET = [2000, 2001]
# define empty errors list
errors = []

In [37]:
# set up progress bar as start of outer loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc = 'YEARS', position=0):
    
    # select JSON_FILE name and check if file exists
    JSON_FILE = f'Data/tmdb_api_results_{YEAR}.json'
    # check if JSON_FILE already exists or not
    file_exists = os.path.isfile(JSON_FILE)
    
    # if file does not exist, create it
    if file_exists == False:
        # save empty dict with just 'imdb_id' to the new json file
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
            
    # save new year as the current df
    df = basics.loc[basics['startYear']==YEAR].copy()
    # save movie ids to list
    movie_ids = df['tconst'].copy()
    
    # load existing data from json to dataframe
    previous_df = pd.read_json(JSON_FILE)
    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    
    # get index and movie id from list
    # INNER LOOP
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                 desc=f'Movies from {YEAR}',
                                 position=1,
                                 leave=True):
        try:
            # retrieve then data for the movie id
            temp = get_movie_with_rating(movie_id)
            # append/extend results to existing file using user defined functions
            write_json(temp, JSON_FILE)
            # short 20ms sleep to prevent overwhelming server
            time.sleep(.02)
            
        except Exception as e:
            # append errors to errors list
            errors.append([movie_id,e])
            
    # convert json file to .csv.gz
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f'Data/final_tmdb_data_{YEAR}.csv.gz', 
                         compression='gzip',
                        index=False)
    
print(f'- Total errors: {len(errors)}')

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1434 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1549 [00:00<?, ?it/s]

- Total errors: 456


## Extract data from 2010 - 2020

In [20]:
YEARS_TO_GET = list(range(2010, 2021))
# define empty errors list
errors = []

In [22]:
basics = pd.read_csv('Data/title_basics.csv.gz')

In [23]:
# set up progress bar as start of outer loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc = 'YEARS', position=0):
    
    # select JSON_FILE name and check if file exists
    JSON_FILE = f'Data/tmdb_api_results_{YEAR}.json'
    # check if JSON_FILE already exists or not
    file_exists = os.path.isfile(JSON_FILE)
    
    # if file does not exist, create it
    if file_exists == False:
        # save empty dict with just 'imdb_id' to the new json file
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
            
    # save new year as the current df
    df = basics.loc[basics['startYear']==YEAR].copy()
    # save movie ids to list
    movie_ids = df['tconst'].copy()
    
    # load existing data from json to dataframe
    previous_df = pd.read_json(JSON_FILE)
    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    
    # get index and movie id from list
    # INNER LOOP
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                 desc=f'Movies from {YEAR}',
                                 position=1,
                                 leave=True):
        try:
            # retrieve then data for the movie id
            temp = get_movie_with_rating(movie_id)
            # append/extend results to existing file using user defined functions
            write_json(temp, JSON_FILE)
            # short 20ms sleep to prevent overwhelming server
            time.sleep(.02)
            
        except Exception as e:
            # append errors to errors list
            errors.append([movie_id,e])
            
    # convert json file to .csv.gz
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f'Data/final_tmdb_data_{YEAR}.csv.gz', 
                         compression='gzip',
                        index=False)
    
print(f'- Total errors: {len(errors)}')

YEARS:   0%|          | 0/11 [00:00<?, ?it/s]

Movies from 2010:   0%|          | 0/3831 [00:00<?, ?it/s]

KeyboardInterrupt: 